# jupyterlite

> [`jupyterlite`](https://github.com/jupyterlite/jupyterlite) provides a statically-hostable JupyterLab and Notebook experience, entirely running in the browser. 

In [ ]:
GH_ORG = NAME = "jupyterlite"
JS_NS = "@jupyterlite"
PY_PROVIDES = [NAME, "jupyterlite_core", "jupyterlite_javascript_kernel"]
URL = f"https://github.com/{GH_ORG}/{NAME}"

In [ ]:
import importnb

with importnb.Notebook():
    from jupyak.tasks import (
        __jupyter_core,
        __jupyter_server,
        __jupyterlab,
        __lumino,
        __notebook,
        __traitlets,
    )
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

## schema

Exclude some packages by npm name from linking. These manifest during webpack builds as:

```bash
[webpack-cli] Error: Cannot find module '@jupyterlab/nbconvert-css/package.json'
```

In [ ]:
lab_excludes = "|".join(
    [
        "celltags-extension",
        "debugger(-extension)?",
        "extensionmanager(-extension)?",
        "hub-extension",
        "mermaid-extension",
        "nbconvert-css",
        "pluginmanager-extension",
        "terminal(-extension)?",
        "template",
    ],
)
nb_excludes = "|".join(
    [
        "documentsearch-extension",
        "lab-extension",
        "terminal-extension",
    ],
)
js_dep_exclude_patterns = [
    rf"^@jupyterlab/({lab_excludes})$",
    rf"^@jupyter-notebook/({nb_excludes})$",
]

In [ ]:
jsk = "./py/jupyterlite-javascript-kernel"

In [ ]:
jsk_tsbi = f"{jsk}/{W.TSCONFIG_BUILDINFO}"
jsk_ext_pkg_json = f"{jsk}/jupyterlite_javascript_kernel/labextension/{W.PACKAGE_JSON}"
lite_third_party = "app/build/third-party-licenses.json"
LAB_EXT = {
    f"./{jsk}": {
        "file_dep": [jsk_tsbi],
        "targets": [jsk_ext_pkg_json],
    },
}

In [ ]:
JS_TASKS = {
    "./": [
        {
            "name": "build",
            "actions": [["doit", "build:py:jupyterlite-core:pre:app"]],
            "file_dep": [],
            "targets": [jsk_tsbi, lite_third_party],
        },
    ],
}

In [ ]:
DEFAULTS = {
    "name": NAME,
    "github": {"url": URL},
    "js": {
        "dependencies": [
            __jupyterlab.NAME,
            __notebook.NAME,
            __lumino.NAME,
        ],
        "dist_exclude_patterns": ["/app"],
        "link_exclude_patterns": js_dep_exclude_patterns,
        "tasks": JS_TASKS,
    },
    "py": {
        "dependencies": [
            __jupyterlab.NAME,
            __notebook.NAME,
            __jupyter_server.NAME,
            __jupyter_core.NAME,
            __traitlets.NAME,
        ],
        "pyproject_tomls": [
            "./py/jupyterlite/pyproject.toml",
            "./py/jupyterlite-core/pyproject.toml",
            "./py/jupyterlite-javascript-kernel/pyproject.toml",
        ],
        "lab_extensions": LAB_EXT,
    },
    "lite": {
        "needs_pth": ["jupyterlite_core"],
    },
}

In [ ]:
@Y.Yak.repo(
    NAME,
    tuple(DEFAULTS["py"]["dependencies"] + DEFAULTS["js"]["dependencies"]),
)
def default_factory(yak: Y.Yak):
    return DEFAULTS